# **This code extracts data from portfolio file (ideally in pdf format) and latest trading data from Dhaka Stock Exchange Website, then portrays the current position of the portfolio. The scenario is updated every 30 seconds for the sake of continuous monitoring. **



In [1]:
# check if bdshare pypdf and tabula are installed if not install
try:
    from bdshare import get_current_trade_data
except:
    !pip install bdshare
    from bdshare import get_current_trade_data

try:
    import tabula
except:
    !pip install tabula-py
    import tabula

# import required modules
import pandas as pd
import numpy as np

# set the number format
pd.options.display.float_format = '{:.2f}'.format

# set pandas display width
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 14)
pd.set_option("display.width", 1000)

# get the data
df_org = tabula.read_pdf("10.pdf", multiple_tables=True, pages="all")[0]
port = df_org.iloc[1:-2, :]
req_cols = ["Company", "Total", "Average", "Total Cost"]
port = port[req_cols]
port["Total Cost"] = port["Total Cost"].str.replace(",", "")
cols = ["Total", "Average", "Total Cost"]
port[cols] = port[cols].apply(pd.to_numeric, errors="coerce")
port.columns = ["TRADING.CODE", "Total Quantity", "Average Cost", "Total Cost"]

# Get the instruments under portfolio
instruments = port["TRADING.CODE"]
instruments = instruments.tolist()

# function for extracting the data
def dse_current(instruments):
    df = []

    for instruments in instruments:
        stock_data = get_current_trade_data(instruments)
        df.append(stock_data)

    df = pd.concat(df)
    df = df.sort_values(by=["symbol"])
    df = df[
        ["symbol", "ltp", "high", "low", "close", "ycp", "trade", "value", "volume"]
    ]
    df = df.reset_index()
    df = df.drop(columns=["index"])
    df.columns = [
        "TRADING.CODE",
        "LTP",
        "HIGH",
        "LOW",
        "CLOSEP",
        "YCP",
        "TRADE",
        "VALUE",
        "VOLUME",
    ]
    cols = df.columns.drop(["TRADING.CODE"])
    df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")

    # merge the data with uploaded portfolio
    df = pd.merge(port, df, on="TRADING.CODE")

    # get current gain/loss
    if (df["CLOSEP"].sum()==0):
      df["Current_Val"] = np.where(df["LTP"] > 0, df["Total Quantity"] * df["LTP"], df["Total Quantity"] * df["YCP"])
    else: 
      df["Current_Val"] = df["Total Quantity"] * df["CLOSEP"] 

    df["Gain/Loss"] = df["Current_Val"] - df["Total Cost"]

    # generate the summary table
    total_quan = df["Total Quantity"].sum()
    total_cost = df["Total Cost"].sum()
    mkt_val = df["Current_Val"].sum()
    gain = df["Gain/Loss"].sum()
    summary = [total_quan, total_cost, mkt_val, gain]
    summary_tab = pd.DataFrame(np.array(summary).reshape(1, 4), columns = ["Total Shares", "Total Cost", "Total Market Value", "Total Gain/Loss"])

    return (df, summary_tab)

Got stderr: Sep 27, 2020 3:22:38 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Sep 27, 2020 3:22:38 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [2]:
# repeat the function after the given interval
from datetime import datetime
import time
from IPython.display import clear_output
from pytz import timezone

bd = timezone("Asia/Dhaka")

def periodic_position(interval):
    while True:
        current_pos = dse_current(instruments)[0]
        summary_tab = dse_current(instruments)[1]
        clear_output(wait=True)
        print("As of:")
        print(datetime.now(bd).strftime("%H:%M:%S.%f - %b %d %Y"))
        print("---")
        print(current_pos)
        print("---")
        #print("Current Total Gain/Loss is: {:2f}".format(gain))
        print(summary_tab)
        time.sleep(interval)

In [ ]:
# now keep running the loop
periodic_position(30)

As of:
21:23:52.956165 - Sep 27 2020
---
   TRADING.CODE  Total Quantity  Average Cost  Total Cost    LTP   HIGH    LOW  CLOSEP    YCP  TRADE  VALUE   VOLUME  Current_Val   Gain/Loss
0      BANKASIA          800405         18.60 14889729.66  18.20  18.20  17.50   17.90  17.50    106   2.94   166349  14327249.50  -562480.16
1      CITYBANK           98424         22.11  2176313.03  24.80  24.90  24.00   24.50  24.40    778  63.86  2609440   2411388.00   235074.97
2           DBH           25000         94.29  2357227.50  93.30  93.60  92.60   93.00  92.70   1309  98.63  1061674   2325000.00   -32227.50
3           EBL          191080         36.82  7034935.88  36.00  36.70  36.00   36.10  36.60     47   1.21    33424   6897988.00  -136947.88
4    JAMUNABANK         3017266         17.98 54247333.93  17.50  17.70  17.40   17.60  17.50    198  14.86   846757  53103881.60 -1143452.33
5    MERCANBANK         2542642         12.94 32909199.86  12.50  12.60  12.30   12.50  12.30    286  25.70